# Momentum Model

## Rules
- Trading is only done monthly.
- Only ETFs in the custom index will be considered.
- Momentum slope will be calculated using 125 days.
- Top N ETFs will be selected.
- Weights will be calculated for these N stocks according to inverse volatility.
- Volatility is calculated using 20 day standard deviation.
- Trend filter calculated based on 200 day average of the index
- If the trend filter is positive, we are allowed to buy.
- Minimum required momentum value is set to 40.
- For each of the N selected ETFs, if the ETF has a momentum value higher than 40, we buy it. If not, we leave that calculated weight for that ETF in cash.
- We sell ETFs if they fall below the minimum required momentum value, or if they leave the index.
- Each month we rebalance and repeat.

In [ ]:
import zipline
from zipline.api import order_target_percent, symbol, set_commission, \
set_slippage, schedule_function, date_rules, time_rules, get_datetime
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import pyfolio as pf
import pandas as pd
import numpy as np
from scipy import stats
from zipline.finance.commission import PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage

"""
    Model Settings
"""
initial_portfolio = 100000
momentum_window = 125
minimum_momentum = 40
portfolio_size = 30
vola_window = 20

"""
    Commission and Slippage Settings
"""
enable_commission = True
commission_pct = 0.001
enable_slippage = True
slippage_volume_limit = 0.025
slippage_impact = 0.05

### Momentum Ranking
Momentum score calculations

In [ ]:
# Exponential Regression
def momentum_score(ts):
    """
        Input: Price time series
        Output: Annualized exponential regression slope, multiplied by R2 
    """
    # list of consecutive numbers
    x = np.arange(len(ts))
    # logs
    log_ts = np.log(ts)
    # regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    # fitnes
    score = annualized_slope * (r_value ** 2)
    return score

#### Position Allocation

Volaitlity based allocation is most common in the professional, quantitaive side of industry. To take on an approximate EQUAL RISK, look at how volatile they are and allocate accordingly. More volatile, smaller weight. Use daily percentage change as basis for calculating standard deviation, a proxy for volatility. A smoothing factor, such as exponentially weighted moving average of standard deviation can be added.

In [ ]:
def volatility(ts):
    return ts.pct_change().rolling(vola_window).std().iloc[-1]

#### Momementum Model Logic

Stock selection based on the momentum score using a time window of 125 days, roughly representing half a year. At the start of backtest, rank stock based on momentum score and buy top 30 in the list with momentum score > 40. Keep position as long as momentum score > 40 at monthly rebalancing, and don't replace all stocks with current top list each month.

#### Downside Protection

Momentum strategies tend to suffer in bear markets. Use an index level trend filter and not allow any new buys when index is below the 200 day Moving Average. Such a trend filter introduces market timing strategy to momentum strategy. Trend filter is not used in code below.

The use of minimum required momentum score will scale portfolio out of market in times of distress, like bear market. Theoretically be anywhere from 100% invested to holding only cash. Instead of leaving in cash buy fixed income ETF, for example, 1-3 year treasury ETF (SHY) for minimal risk or 20+ year treasury ETF (TLT) for litle more price risk.

In [ ]:
# output percentage return for previous month
def output_progress(context):
    """
        Output performance numbers during backtest run.
        prints out past month's performance while backtest run
    """
    # today's date
    today = zipline.api.get_datetime().date()
    
    # percentage difference since last month
    perf_pct = (context.portfolio.portfolio_value / context.last_month) - 1
    
    # print performance
    print("{} - Last month result: {:.2%}".format(today, perf_pct))
    
    # remember today's portfolio value for next month calc
    context.last_month = context.portfolio.portfolio_value

In [ ]:
"""
    Initialization and trading logic
"""
def initialize(context):
    # set commission and slippage
    if enable_commission:
        comm_model = PerDollar(cost = commission_pct)
    else:
        comm_model = PerDollar(cost = 0.0)
    set_commission(comm_model)
    
    if enable_slippage:
        slippage_model = VolumeShareSlippage(volume_limit = slippage_volume_limit, price_impact = slippage_impact)
    else:
        slippage_model = slippage_model = FixedSlippage(spread = 0.0)
    set_slippage(slippage_model)
    
    # Used only for progress output
    context.last_month = initial_portfolio
    
    # Store index membership
    context.index_members = pd.read_csv('../resources/data/index_members/sp500.csv', 
                                        index_col = 0, parse_dates = [0])
    
    # Schedule rebalance monthly
    schedule_function(func = rebalance, 
                      date_rule = date_rules.month_start(), 
                      time_rule = time_rules.market_open()
                     )

In [ ]:
def rebalance(context, data):
    # progress output during backtest
    output_progress(context)
        
    # which stocks were in index on trading day
    # get date of trading in backtest
    today = zipline.api.get_datetime()
    
    # get index makeup of all days prior to trading day
    all_prior = context.index_members.loc[context.index_members.index < today]
    
    # retrieve first column of last, i.e. latest entry
    latest_day = all_prior.iloc[-1, 0]
    
    # split text string with tickers into a list
    list_of_tickers = latest_day.split(',')
    todays_universe = [symbol(ticker) for ticker in list_of_tickers]
    
    # get historical data
    hist = data.history(todays_universe, "close", momentum_window, "1d")
    
    # momentum ranking table
    ranking_table = hist.apply(momentum_score).sort_values(ascending = False)
    
    """
        Sell Logic
            check if existing position should be sold
            1. stock not part of index
            2. stock too low of momentum value    
    """
    kept_positions = list(context.portfolio.positions.keys())
    for security in context.portfolio.positions:
        if (security not in todays_universe):
            order_target_percent(security, 0.0)
            kept_positions.remove(security)
        elif ranking_table[security] < minimum_momentum:
            order_target_percent(security, 0.0)
            kept_positions.remove(security)
            
    """
        Stock Selection Logic
            check how many stocks are being kept from last month.
            fill from top of ranking list until desired total number
            of desired portfolio holdings.
    """
    replacement_stocks = portfolio_size - len(kept_positions)
    buy_list = ranking_table.loc[~ranking_table.index.isin(kept_positions)][:replacement_stocks]
    
    new_portfolio = pd.concat((buy_list, ranking_table.loc[ranking_table.index.isin(kept_positions)]))
    
    """
        Inverse volatility for stocks, and target position weights
    """
    vola_table = hist[new_portfolio.index].apply(volatility)
    inv_vola_table = 1 / vola_table
    sum_inv_vola = np.sum(inv_vola_table)
    vola_target_weights = inv_vola_table/sum_inv_vola
    
    for security, rank in new_portfolio.iteritems():
        weight = vola_target_weights[security]
        if security in kept_positions:
            order_target_percent(security, weight)
        else:
            if ranking_table[security] > minimum_momentum:
                order_target_percent(security, weight)

In [ ]:
def analyze(context, perf):
    perf['max'] = perf.portfolio_value.cummax()
    perf['dd'] = (perf.portfolio_value / perf['max']) - 1
    maxdd = perf['dd'].min()
    
    ann_ret = (np.power((perf.portfolio_value.iloc[-1] / perf.portfolio_value.iloc[0]), (252/len(perf)))) - 1
    
    print("Annualized Return: {:.2%} Max Drawdown: {:.2%}".format(ann_ret, maxdd))
    
    # Use PyFolio for performance
    returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
    pf.create_full_tear_sheet(returns, positions = positions, transactions = transactions,
                          round_trips = True)
    
    return

In [ ]:
start = datetime(1997, 1, 1, 8, 15, 12, 0, pytz.UTC)
end = datetime(2018, 12, 31, 8, 15, 12, 0, pytz.UTC)
perf = zipline.run_algorithm(
    start = start, end = end, 
    initialize = initialize, analyze = analyze, 
    capital_base = initial_portfolio, 
    data_frequency = 'daily', 
    bundle = 'random_stock_data')